In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.externals import joblib

sys.path.insert(1, '..')

In [6]:
from sklearn.ensemble import GradientBoostingClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, LassoCV
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [3]:
from xgboost import XGBClassifier

/Users/dkn22/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
data_dir = 'data/external/'
train_df = pd.read_csv(data_dir + 'train.csv')
test_df = pd.read_csv(data_dir + 'test.csv')
sample_submission = pd.read_csv(data_dir + 'sample_submission.csv')

In [9]:
test_ids = test_df['id'].values
target_cols = [y for y in train_df.columns if y not in ('id', 'comment_text')]
targets = train_df[target_cols]

In [10]:
tfidf_corpus = joblib.load('data/processed/tfidf_corpus.pickle')

In [11]:
train_text = tfidf_corpus[:train_df.shape[0]]
test_text = tfidf_corpus[train_df.shape[0]:]

In [32]:
lasso = LogisticRegressionCV(penalty='l1', scoring='roc_auc', solver='saga')
xgboost = XGBClassifier(n_estimators=1000)

In [34]:
pipeline = Pipeline([
    ('feature_selection', SelectFromModel(MultiOutputClassifier(lasso))),
    ('classification', ClassifierChain(xgboost))
        ])


In [35]:
pipeline.fit(train_text, targets)

/Users/dkn22/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
